In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from collections import defaultdict
import dateutil.parser
from pprint import pprint
import pandas
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from copy import deepcopy
import seaborn as sns
import urllib2, sys
import string
import gevent.monkey
gevent.monkey.patch_socket()
from gevent.pool import Pool

%matplotlib inline

### Helper methods for BeautifulSoup ###

In [4]:
def get_soup_ps(prefix, suffix):
    response = requests.get(prefix + suffix)   
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    return soup

def get_soup_url(url):
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    return soup

### Helper methods to gather data on given movie ###

In [20]:
def get_movie_value(soup, field_name):
    '''Grab a value from boxofficemojo HTML
    
    Takes a string attribute of a movie on the page and
    returns the string in the next sibling object
    (the value for that attribute)
    or None if nothing is found.
    '''
    obj = soup.find(text=re.compile(field_name))
    if not obj: 
        return None
    # this works for most of the values
    next_sibling = obj.findNextSibling()
    if next_sibling:
        return next_sibling.text 
    else:
        next_sibling = obj.find_parent().findNextSibling()
        return next_sibling

    
def get_movie_values_odd(soup, field_name):
    '''Grabs value from boxofficemojo HTML in 
    Domestic Summary box
    
    Takes a string attribute of a movie on the page and
    returns the string in the next sibling object
    (the value for that attribute)
    or None if nothing is found.
    '''
    try:
        div_with_stats = soup.find_all('div',class_='mp_box_content')[1]
        flag = False
        obj = ''
        for element in div_with_stats.find_all('td'):
            if flag:
                obj = element.text
                return obj
            if field_name in str(element):
                flag = True
    except:
        return None

            
def get_weekly_rev(soup):
    '''Retrieves weekly revenue for given movie
    
    Returns a list of the weekly revenue,
    ordered by week since release
    '''
    try:
        weekly_rev_list = []
        for element in soup.find_all('center')[1]:
            box = element.find_all('font')
            for thing in box:
                if '$' in thing.text:
                    clean_num = money_to_int(thing.text)
                    weekly_rev_list.append(clean_num)
        if weekly_rev_list[0] == weekly_rev_list[1]:
            return weekly_rev_list[3::3]
        else:
            return weekly_rev_list[1::3]
    except:
        return None
    

def to_date(datestring):
    try:
        date = dateutil.parser.parse(datestring)
        return date
    except:
        return datestring
    

def money_to_int(moneystring):
    try:
        moneystring = moneystring.replace('$', '').replace(',', '')
        if 'mil' in moneystring:
            moneystring_digits = moneystring.split(' ')[0].strip()
            return int(moneystring_digits) * 1000000
        return int(moneystring)
    except:
        return None
    
    
def runtime_to_minutes(runtimestring):
    try:
        runtime = runtimestring.split()
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        return None

    
def theaters_to_int(theaterstring):
    try:
        theaters = int(str(theaterstring.split()[0]).replace(',',''))
        return theaters
    except:
        return None


def release_time_to_days(releasestring):
    try:
        days = int(releasestring.split()[0])
        return days
    except:
        return None

## Gather the links to the top movies of all time ##

In [6]:
movie_prefix = 'http://www.boxofficemojo.com/alltime/domestic.htm?page=' 
partial_movie_suffix = '&p=.htm'
top_movie_links = set()

for page in range(1,41):
    full_movie_suffix = str(page) + partial_movie_suffix
    soup = get_soup_ps(movie_prefix, full_movie_suffix)
    for element in soup.find_all('a'):
        link = element.get('href')
        if link.startswith('/movies/?id'):
            top_movie_links.add(link)
    if '/movies/?id=michelledarnell.htm' in top_movie_links:
        top_movie_links.remove('/movies/?id=michelledarnell.htm')

In [7]:
# Number of movies in database
print len(top_movie_links)

3903


In [17]:
# Smaller set of movies to test data collection on
test_movie_links = {x for x in top_movie_links if 'planes' in x}
print test_movie_links

set([u'/movies/?id=planes.htm', u'/movies/?id=planes2.htm', u'/movies/?id=planestrainsandautomobiles.htm'])


## Collect data from BoxOfficeMojo ##

Output is a list of dictionaries containing all data for a single movie

In [25]:
headers = ['movieid','movie title', 'domestic total gross',
           'release date', 'runtime (mins)', 'rating', 
           'genre', 'production budget', 'distributor',
            'opening weekend revenue', 'number of theaters',
            'time in theaters (days)']

base_url = 'http://www.boxofficemojo.com/'

movie_data = []
rev_data = {}
missing_links = []

counter = 1

time_count = 0

for movie in list(top_movie_links):
    try:
        if time_count % 50 == 0: print time_count
        time_count += 1

        # Create a movieid
        movieid = counter
        counter += 1


        # Create dictionary of movieid to list of weekly revenues
        movie_link_split = movie.split('?')
        weekly_suffix = movie_link_split[0] + '?page=weekly&' + movie_link_split[1]
        weekly_soup = get_soup_ps(base_url, weekly_suffix)

        weekly_revenue = get_weekly_rev(weekly_soup)

        rev_data[movieid] = weekly_revenue


        # Create list of movie data dictionarys
        soup = get_soup_ps(base_url, movie)
        adj_soup = get_soup_ps(base_url, movie + '&adjust_yr=2016&p=.htm')

        raw_title_string = soup.find('title').text
        title = raw_title_string.split('(')[0].strip()

        raw_release_date = get_movie_value(soup,'Release Date')
        release_date = to_date(raw_release_date)

        raw_domestic_total_gross = get_movie_value(adj_soup,'Domestic Total')
        domestic_total_gross = money_to_int(raw_domestic_total_gross)

        raw_runtime = get_movie_value(soup,'Runtime')
        runtime = runtime_to_minutes(raw_runtime)

        rating = get_movie_value(soup,'MPAA Rating')

        genre = get_movie_value(soup, 'Genre:')

        raw_prod_budget = get_movie_value(soup, 'Production Budget')
        prod_budget = money_to_int(raw_prod_budget)

        distributor = get_movie_value(soup, 'Distributor:')

        raw_opening_weekend = get_movie_values_odd(adj_soup, 'Opening')
        opening_weekend = money_to_int(raw_opening_weekend)

        raw_widest_release = get_movie_values_odd(soup, 'Widest')
        widest_release = theaters_to_int(raw_widest_release)

        raw_realease_time = get_movie_values_odd(soup, 'In Release')
        release_time = release_time_to_days(raw_realease_time)

        raw_close_date = get_movie_values_odd(soup, 'Close')
        close_date = to_date(raw_close_date)

        # Calculate days in theaters if not given
        if release_time == None and close_date != None:
            day_diff = close_date - release_date
            release_time = release_time_to_days(str(day_diff))
        elif release_time == None and close_date == None and weekly_revenue != None:
            num_weeks = len(weekly_revenue)
            release_time = num_weeks * 7



        movie_dict = dict(zip(headers, [movieid, 
                                        title,
                                        domestic_total_gross,
                                        release_date,
                                        runtime,
                                        rating,
                                        genre,
                                        prod_budget,
                                        distributor,
                                        opening_weekend,
                                        widest_release,
                                        release_time]))

        movie_data.append(movie_dict) 
    
    except:
        print 'ChunkedEncodingError'
        missing_links.append(movie)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900


In [27]:
test_rev_data = {k: rev_data[k] for k in rev_data.keys()[:10]}
print (test_rev_data)
print (movie_data[:10])

{1: [4934, 5692, 5092, 3050, 2618, 1822, 1592, 1251], 2: [10639, 9349, 9011, 7849, 5817, 4080, 3556, 2798, 1903, 1726], 3: [5325, 2944, 1842, 1211, 759, 763, 607, 465, 430, 397, 420, 485, 691, 480, 421], 4: [6324, 3495, 2855, 1455, 1200, 1055, 650, 605, 580], 5: [7924, 5851, 3130, 2759, 4243, 3014, 2248, 1730, 1349, 1196], 6: [43292, 41573, 5905, 4385, 3507, 3075, 2329], 7: [7709, 4813, 2411, 1659, 1120, 930, 794, 659, 2182, 1192, 1008, 1519, 1022, 1548], 8: [2181, 2530, 1190, 1172], 9: [7463, 6550, 7799, 5877, 3927, 2917, 2316, 1860, 1667], 10: [12665, 7955, 4965, 2985, 2017, 1999, 1181, 1037]}
[{'rating': u'PG', 'time in theaters (days)': 56, 'movieid': 1, 'production budget': None, 'domestic total gross': 90944400, 'number of theaters': 1515, 'opening weekend revenue': 8129500, 'distributor': u'Orion Pictures', 'movie title': u'Dirty Rotten Scoundrels', 'genre': u'Comedy', 'release date': datetime.datetime(1988, 12, 16, 0, 0), 'runtime (mins)': 110}, {'rating': u'R', 'time in theate

In [28]:
print len(missing_links)

0


In [29]:
# Check that the data is all there
print len(rev_data)
print len(movie_data)

# Sanity check what the longest list is
max_count = 0
for movid,weekly_list in rev_data.items():
    try:
        count = len(weekly_list)
        if count > max_count:
            max_count = count
    except:
        continue
print max_count

3903
3903
671


In [ ]:
# with open('movie_data2.pickle', 'wb') as handle:
#     pickle.dump(movie_data, handle)

# with open('rev_data2.pickle', 'wb') as handle:
#     pickle.dump(movie_data, handle)

In [31]:
# with open('movie_data3.pickle', 'wb') as handle:
#     pickle.dump(movie_data, handle)

# with open('rev_data3.pickle', 'wb') as handle:
#     pickle.dump(rev_data, handle)

# Input movie data into pandas DataFrame #

In [32]:
moviedf = pd.DataFrame(movie_data)

### Clean data ###

In [33]:
print moviedf.dtypes

distributor                        object
domestic total gross              float64
genre                              object
movie title                        object
movieid                             int64
number of theaters                float64
opening weekend revenue           float64
production budget                 float64
rating                             object
release date               datetime64[ns]
runtime (mins)                    float64
time in theaters (days)           float64
dtype: object


In [34]:
moviedf.count(axis=0)

distributor                3900
domestic total gross       3900
genre                      3900
movie title                3903
movieid                    3903
number of theaters         3632
opening weekend revenue    3297
production budget          2025
rating                     3900
release date               3900
runtime (mins)             3899
time in theaters (days)    3654
dtype: int64

In [36]:
moviedf.describe()

,domestic total gross,movieid,number of theaters,opening weekend revenue,production budget,runtime (mins),time in theaters (days)
count,3.900000e+03,3903.000000,3632.000000,3.297000e+03,2.025000e+03,3899.000000,3654.000000
mean,9.625383e+07,1952.000000,2230.817181,2.354510e+07,5.487065e+07,108.427289,93.705255
std,9.894027e+07,1126.843379,933.700176,2.306523e+07,4.632437e+07,19.376241,175.692769
min,1.263930e+07,1.000000,2.000000,4.960000e+04,1.500000e+04,38.000000,0.000000
25%,3.758938e+07,976.500000,1489.750000,1.054660e+07,2.200000e+07,95.000000,42.000000
50%,6.257970e+07,1952.000000,2292.500000,1.649820e+07,4.000000e+07,106.000000,77.000000
75%,1.153584e+08,2927.500000,2950.000000,2.766540e+07,7.500000e+07,118.000000,112.000000
max,1.239014e+09,3903.000000,4468.000000,2.479667e+08,3.000000e+08,227.000000,4697.000000


In [37]:
moviedf[moviedf['time in theaters (days)'] > 500]

,distributor,domestic total gross,genre,movie title,movieid,number of theaters,opening weekend revenue,production budget,rating,release date,runtime (mins),time in theaters (days)
28,Warner Bros.,40405900.0,IMAX,Under the Sea 3D,29,108.0,771100.0,NaN,G,2009-02-13,40.0,2310.0
490,Sony Classics,22447900.0,IMAX,Across the Sea of Time,491,2.0,NaN,NaN,G,1995-10-20,51.0,3142.0
666,Warner Bros.,15663600.0,IMAX,To the Arctic,667,52.0,289500.0,NaN,G,2012-04-20,40.0,1127.0
792,IMAX,126001600.0,IMAX,Space Station 3-D,793,64.0,733000.0,NaN,Unrated,2002-04-19,46.0,4697.0
1105,Sony Classics,24348500.0,IMAX,Cirque du Soleil: Journey of Man,1106,22.0,80400.0,NaN,G,2000-05-05,38.0,1402.0
1187,WGB.,23353700.0,IMAX,Shackleton's Antarctic Adventure,1188,26.0,NaN,NaN,G,2001-02-10,40.0,1034.0
1229,IMAX,82589700.0,IMAX,T-Rex: Back to the Cretaceous,1230,38.0,NaN,NaN,Unrated,1998-10-23,45.0,3234.0
1317,IMAX,26204200.0,IMAX,Galapagos,1318,14.0,NaN,NaN,Unrated,1999-10-29,40.0,3794.0
1621,Buena Vista,24491000.0,IMAX,Ghosts of the Abyss,1622,97.0,2032100.0,NaN,G,2003-04-11,61.0,917.0
1664,Warner Bros.,57619400.0,IMAX,Deep Sea 3D,1665,47.0,930100.0,NaN,G,2006-03-03,40.0,2842.0


In [ ]:
# Cell to look at outliers
# m = moviedf[moviedf['movieid'] == 792]
# print m
moviedf = moviedf[moviedf['movie title'] != 'Red Dawn']

In [ ]:
moviedf[18:19]

In [ ]:
moviedf.describe()

In [ ]:
moviedf.head()

## Adding Additional Features ##

In [38]:
moviedf['release month'] = moviedf['release date'].dt.month

In [39]:
moviedf.head()

,distributor,domestic total gross,genre,movie title,movieid,number of theaters,opening weekend revenue,production budget,rating,release date,runtime (mins),time in theaters (days),release month
0,Orion Pictures,90944400.0,Comedy,Dirty Rotten Scoundrels,1,1515.0,8129500.0,NaN,PG,1988-12-16,110.0,56.0,12.0
1,Fox,252358600.0,Action Thriller,Speed,2,2169.0,30088300.0,30000000.0,R,1994-06-10,116.0,70.0,6.0
2,Paramount,48884600.0,Comedy,The Out-of-Towners,3,2128.0,14084800.0,75000000.0,PG-13,1999-04-02,90.0,105.0,4.0
3,Universal,39263600.0,War Romance,Captain Corelli's Mandolin,4,1612.0,11081500.0,57000000.0,R,2001-08-17,129.0,63.0,8.0
4,Paramount,108640100.0,Comedy,"Planes, Trains and Automobiles",5,1684.0,15596500.0,NaN,R,1987-11-25,92.0,70.0,11.0


### Input revenue data into pandas DataFrame ###

In [40]:
revdf = pd.DataFrame.from_dict({k : pd.Series(v) for k,v in rev_data.items()}, 
                                     orient='index')

In [41]:
revdf.head()

,0,1,2,3,4,5,6,7,8,9,...,661,662,663,664,665,666,667,668,669,670
1,4934,5692.0,5092.0,3050.0,2618.0,1822.0,1592.0,1251.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10639,9349.0,9011.0,7849.0,5817.0,4080.0,3556.0,2798.0,1903.0,1726.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5325,2944.0,1842.0,1211.0,759.0,763.0,607.0,465.0,430.0,397.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6324,3495.0,2855.0,1455.0,1200.0,1055.0,650.0,605.0,580.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7924,5851.0,3130.0,2759.0,4243.0,3014.0,2248.0,1730.0,1349.0,1196.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
index_list = revdf.index.tolist()
print index_list[:100]

In [ ]:
# Convert items to floats
rev_list = []
iterator = revdf.iterrows()
for item in iterator:
    num_list = []
    for num in item[1]:
        money = money_to_int(str(num))
        num_list.append(money)
    rev_list.append(num_list)

In [ ]:
revdf = pd.DataFrame(rev_list, index=index_list)

In [ ]:
revdf.head()

In [ ]:
print revdf.index.tolist()[:100]

In [ ]:
revdf.to_csv('csv/init_float_revdf.csv',encoding='utf-8')

In [ ]:
# revdf_nulls = revdf[revdf.isnull().all(axis=1)]
# revdf_nulls.head()

In [ ]:
# weekly_rev_df.drop(791, inplace=True)

In [ ]:
# weekly_rev_df.dropna(axis=1, how='all', inplace=True)

In [ ]:
# weekly_rev_df.dropna(axis=0, how='all', inplace=True)

In [ ]:
# Save DFs
# moviedf.to_csv('csv/moviedf2.csv',encoding='utf-8')
# weekly_rev_df.to_csv('csv/weekly_rev_df1.csv',encoding='utf-8')

In [ ]:
# Includes the movie indexes to include, where weekly revenue is not null
# index_list = weekly_rev_df.index.tolist()
# print index_list[:100]

In [ ]:
# moviedf.index

In [ ]:
# moviedf_clean = moviedf[moviedf.index.isin(index_list)]

In [ ]:
# movie_index_list = moviedf_clean.index.tolist()
# print movie_index_list[:100]

In [ ]:
# print movie_index_list == index_list

In [ ]:
# print len(movie_index_list)
# print len(index_list)

In [ ]:
print len(moviedf.index.tolist())
print len(revdf.index.tolist())

In [ ]:
revdf.head()

In [ ]:
revdf_exnullrows = revdf.dropna(axis=1, how='all')

In [ ]:
print len(revdf_exnullrows)
revdf_exnullrows.head()

In [42]:
ittr = revdf.iterrows()

decay_rates = []
for row in ittr:
    rev_series = row[1]
    a = rev_series[0]
    t = rev_series.count()
    y = rev_series[t-1]
    k = (np.log(y) - np.log(a)) / t
    decay_rates.append(k)

In [43]:
print decay_rates[:10]

[-0.17152584832779882, -0.18187199018729922, -0.16916901045968302, -0.26545323275432192, -0.18909134733078653, -0.41750412421867089, -0.11467319302211353, -0.15526794901616792, -0.16654798604228807, -0.31281379427578093]


In [44]:
revdf['decay rate'] = np.asarray(decay_rates)

In [45]:
revdf.head()

,0,1,2,3,4,5,6,7,8,9,...,662,663,664,665,666,667,668,669,670,decay rate
1,4934,5692.0,5092.0,3050.0,2618.0,1822.0,1592.0,1251.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.171526
2,10639,9349.0,9011.0,7849.0,5817.0,4080.0,3556.0,2798.0,1903.0,1726.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.181872
3,5325,2944.0,1842.0,1211.0,759.0,763.0,607.0,465.0,430.0,397.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.169169
4,6324,3495.0,2855.0,1455.0,1200.0,1055.0,650.0,605.0,580.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.265453
5,7924,5851.0,3130.0,2759.0,4243.0,3014.0,2248.0,1730.0,1349.0,1196.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.189091


In [51]:
len(revdf[revdf['decay rate'] > 0])

87

In [ ]:
revdf_exnullrows['decay rate'] = np.asarray(decay_rates)

In [ ]:
revdf_exnullrows.head()

In [ ]:
y_avgs = revdf_exnullrows.mean(axis=0)
xs = np.arange(len(y_avgs))
plt.plot(xs[:100], y_avgs[:100])

In [ ]:
# Get rid of movies where there are over 100 weeks of revenue data (mostly IMAX movies)
revdf_exoutliers = revdf_exnullrows[revdf_exnullrows.count(axis=1) < 100]
len(revdf_exoutliers)

In [ ]:
moviedf.head()

In [ ]:
test1 = pd.DataFrame({'title': ['a','b','c','d'], 'score': [4,3,3,5],'movieid': [1,2,4,6]})

dd = {1: [11,23,23,2], 2:[121,14,123], 3:[12,21,1]}
test2 = pd.DataFrame.from_dict({k : pd.Series(v) for k,v in dd.items()}, 
                                     orient='index')

In [ ]:
test2['decay rate'] = np.asarray([0.2,-0.6,0.02])
test2

In [ ]:
test1

In [ ]:
# test3 = pd.concat([test2['decay rate'], test2.index], axis=1)
# test3
# result = pd.merge(test1, test2['decay rate'], left_on='movieid', right_on='movieid', how='inner')
# result
test2_short = pd.DataFrame(test2['decay rate'], index=test2.index)
test2_short
test3 = pd.merge(test1, test2_short, left_on='movieid', right_index=True, how='inner')
test3

In [52]:
decaydf = pd.DataFrame(revdf['decay rate'], index=revdf.index)
decaydf.head()

,decay rate
1,-0.171526
2,-0.181872
3,-0.169169
4,-0.265453
5,-0.189091


In [53]:
merged = pd.merge(moviedf, decaydf, left_on='movieid', right_index=True, how='inner')
merged.describe()

,domestic total gross,movieid,number of theaters,opening weekend revenue,production budget,runtime (mins),time in theaters (days),release month,decay rate
count,3.643000e+03,3643.000000,3616.000000,3.242000e+03,1.994000e+03,3642.000000,3643.000000,3643.000000,3643.000000
mean,8.957397e+07,1948.875377,2236.897677,2.372106e+07,5.548542e+07,107.799561,93.986001,6.974197,-0.202691
std,8.587887e+07,1125.529929,928.836871,2.313544e+07,4.625888e+07,18.875513,175.883438,3.447081,0.148253
min,1.263930e+07,1.000000,2.000000,4.960000e+04,1.500000e+04,38.000000,7.000000,1.000000,-1.256936
25%,3.613510e+07,979.000000,1498.000000,1.064488e+07,2.300000e+07,95.000000,42.000000,4.000000,-0.274697
50%,5.966970e+07,1945.000000,2302.500000,1.657520e+07,4.000000e+07,105.000000,77.000000,7.000000,-0.202300
75%,1.095184e+08,2918.500000,2952.250000,2.788838e+07,7.500000e+07,118.000000,112.000000,10.000000,-0.147904
max,1.114286e+09,3903.000000,4468.000000,2.479667e+08,3.000000e+08,227.000000,4697.000000,12.000000,1.143955


In [27]:
merged.head()

NameError: name 'merged' is not defined

In [54]:
merged.to_csv('csv/merged_table_4_20.csv',encoding='utf-8')

# Plot data (density and scatter plots) #

In [ ]:
def plot_kde(df, category):
    dropped = df[category].dropna()
    density = gaussian_kde(dropped)
    xs = np.linspace(dropped.min(),dropped.max(),300)
    ys = density(xs)
    plt.plot(xs, ys)
    plt.title('PDF of ' + category)
    plt.ylabel('Probability')
    plt.xlabel(category)

In [ ]:
plot_kde(merged, 'domestic total gross')

In [ ]:
plot_kde(merged, 'number of theaters')

In [ ]:
plot_kde(merged, 'opening weekend revenue')

In [ ]:
plot_kde(merged, 'production budget')

In [ ]:
plot_kde(merged, 'runtime (mins)')

In [ ]:
plot_kde(merged, 'time in theaters (days)')

In [ ]:
plot_kde(merged, 'release month')

In [ ]:
plot_kde(merged, 'decay rate')

In [ ]:
def plot_bar(df, column):
    ys = df[column].value_counts(normalize=True, sort=False)
    xs = np.arange(len(ys))
    plt.figure(figsize=(18,5))
    plt.bar(xs, ys.values)
    plt.xticks(xs, ys.index, rotation=45)

In [ ]:
plot_bar(merged, 'genre')

In [ ]:
plot_bar(merged, 'rating')

In [ ]:
plot_bar(merged, 'distributor')

In [ ]:
def plot_scatter(df, category):
    xs = df[category]
    ys = df['decay rate']
    plt.scatter(xs, ys)
    plt.savefig('img/scatter'+category+'.png')

In [ ]:
plot_scatter(merged, 'domestic total gross')

In [ ]:
plot_scatter(merged, 'number of theaters')

In [ ]:
plot_scatter(merged, 'opening weekend revenue')

In [ ]:
plot_scatter(merged, 'production budget')

In [ ]:
plot_scatter(merged, 'runtime (mins)')

In [ ]:
plot_scatter(merged, 'time in theaters (days)')

In [ ]:
plot_scatter(merged, 'release month')

In [ ]:
def plot_scatter_categorical(df, category):
    d = {}
    i = 0
    for c in df[category].unique():
        d[c] = i
        i += 1
    copy = df.copy(deep=True)
    copy[category + '_cat'] = copy[category].map(d)
    xs = copy[category + '_cat']
    ys = copy['decay rate']
    plt.scatter(xs, ys)

In [ ]:
plot_scatter_categorical(merged, 'genre')

In [ ]:
plot_scatter_categorical(merged, 'rating')

In [ ]:
plot_scatter_categorical(merged, 'distributor')

In [ ]:
merged.corr()

In [ ]:
merged.head()

# Create models #

In [ ]:
from patsy import dmatrices, dmatrix
from sklearn.cross_validation import train_test_split
import statsmodels.api as sm
from sklearn.metrics import r2_score

In [ ]:
renamed = merged.rename(columns={'distributor':'dist', 'domestic total gross': 'dom_tot_gross',
                                'movie title': 'title', 'number of theaters': 'num_theaters',
                                'opening weekend revenue': 'open_rev', 'production budget': 
                                'budget', 'release date': 'release', 'runtime (mins)': 'runtime',
                                'time in theaters (days)': 'time_theaters', 'release month': 'month',
                                'decay rate': 'decay'})

In [ ]:
renamed.to_csv('csv/analysisdf.csv',encoding='utf-8')

In [ ]:
renamed.head()

In [ ]:
y = renamed['decay']
y.head()

In [ ]:
renamed.drop('decay', axis=1, inplace=True)

In [ ]:
X = renamed

In [ ]:
# Split into reporting (30%) and training (70%)
X_train, X_report, y_train, y_report = train_test_split(X, y, 
                                                        test_size = 0.3,
                                                        random_state = 4444)

# X_train['y_train'] = y_train

In [ ]:
# Split training data into training (60%) and testing (40%) 
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train,
                                                       test_size = 0.4,
                                                       random_state = 4444)

In [ ]:
# Check lengths
print 'Training: ' + str(len(X_train) == len(y_train))
print 'Testing: ' + str(len(X_test) == len(y_test))
print 'Reporting: ' + str(len(X_report) == len(y_report))

In [ ]:
X_train.head()

### First Model: num_theaters, runtime, month, rating, genre ###

In [ ]:
def create_Xy(data, y_category, X_categories, intercept=True):
    formula = y_category + ' ~ '
    for c in X_categories:
        formula = formula + c + ' + '
    formula = formula[:-3]
    if not(intercept):
        formula = formula + ' - 1'
    y, X = dmatrices(formula, data=data, return_type='dataframe')
    return y, X

In [ ]:
features_1 = ['num_theaters','runtime','C(month)','rating','genre']
y1_train, X1_train = create_Xy(X_train,'decay',features_1, intercept=False)
y1_test, X1_test = create_Xy(X_train,'decay',features_1, intercept=False)
y1_report, X1_report = create_Xy(X_train,'decay',features_1, intercept=False)

In [ ]:
model_1 = sm.OLS(y1_train, X1_train)
results_1 = model_1.fit()

In [ ]:
results_1.summary()

In [ ]:
y1_pred_test = results_1.predict(X1_test)

In [ ]:
r2_1 = r2_score(y1_test, y1_pred_test)
r2_1

In [ ]:
adjr2_1 = 1 - (1 - r2_1) * ((len(y1_test) - 1) / (len(y1_test) - len(X1_test.columns) - 1))
adjr2_1

### Second Model: num_theaters, runtime, month, rating

In [ ]:
features_2 = ['num_theaters','runtime','C(month)','rating']
y2_train, X2_train = create_Xy(X_train,'decay',features_2, intercept=False)
y2_test, X2_test = create_Xy(X_train,'decay',features_2, intercept=False)
y2_report, X2_report = create_Xy(X_train,'decay',features_2, intercept=False)

In [ ]:
model_2 = sm.OLS(y2_train, X2_train)
results_2 = model_2.fit()

In [ ]:
results_2.summary()

In [ ]:
y2_pred_test = results_2.predict(X2_test)

In [ ]:
r2_2 = r2_score(y2_test, y2_pred_test)
r2_2

In [ ]:
adjr2_4 = 1 - (1 - r2_4) * ((len(y4_test) - 1) / (len(y4_test) - len(X4_test.columns) - 1))
adjr2_4

### Third Model: num_theaters, runtime, month, genre

In [ ]:
features_3 = ['num_theaters','runtime','C(month)','genre']
y3_train, X3_train = create_Xy(X_train,'decay',features_3, intercept=False)
y3_test, X3_test = create_Xy(X_train,'decay',features_3, intercept=False)
y3_report, X3_report = create_Xy(X_train,'decay',features_3, intercept=False)

In [ ]:
model_3 = sm.OLS(y3_train, X3_train)
results_3 = model_3.fit()

In [ ]:
results_3.summary()

In [ ]:
y3_pred_test = results_3.predict(X3_test)

In [ ]:
r2_3 = r2_score(y3_test, y3_pred_test)
r2_3

In [ ]:
adjr2_3 = 1 - (1 - r2_3) * ((len(y3_test) - 1) / (len(y3_test) - len(X3_test.columns) - 1))
adjr2_3

### Fourth Model: num_theaters, runtime, month

In [ ]:
features_4 = ['num_theaters','runtime','C(month)']
y4_train, X4_train = create_Xy(X_train,'decay',features_4, intercept=False)
y4_test, X4_test = create_Xy(X_train,'decay',features_4, intercept=False)
y4_report, X4_report = create_Xy(X_train,'decay',features_4, intercept=False)

In [ ]:
model_4 = sm.OLS(y4_train, X4_train)
results_4 = model_4.fit()

In [ ]:
results_4.summary()

In [ ]:
y4_pred_test = results_4.predict(X4_test)

In [ ]:
r2_4 = r2_score(y4_test, y4_pred_test)
r2_4

In [ ]:
adjr2_4 = 1 - (1 - r2_4) * ((len(y4_test) - 1) / (len(y4_test) - len(X4_test.columns) - 1))
adjr2_4

## Create Minimum Viable Product Graphs

In [ ]:
# Best model so far is Model 3
# fig = plt.figure(figsize=(10,100))
# fig = sm.graphics.plot_partregress_grid(results_3, fig=fig)

In [ ]:
# fig, ax = plt.subplots(figsize=(10,100))
# fig = sm.graphics.plot_ccpr_grid(results_3, fig=fig)

In [ ]:
# sns.set(style="darkgrid", color_codes=True)
# tips = sns.load_dataset("tips")
# g = sns.jointplot("total_bill", "tip", data=tips, kind="reg",
#                   xlim=(0, 60), ylim=(0, 12), color="r", size=7)

In [ ]:
columns = X3_train.columns.tolist()
xlabels = []
for item in columns[:-2]:
    label = re.search('(?<=\[).*(?=\])', str(item)).group()
    xlabels.append(label)
xlabels.append(columns[-2])
xlabels.append(columns[-1])

formula = 'decay ~ num_theaters + runtime + C(month) + genre - 1'
sns.set_context("poster")
plt.figure(figsize=(20, 20))

sns.coefplot(formula, X_train)
plt.xticks(np.arange(len(columns)),xlabels, rotation=90, fontsize=10)

ax = plt.gca()
pos1 = ax.get_position()
pos2 = [pos1.x0 - 0.05, pos1.y0 + 0.37,  pos1.width + 0.11, pos1.height - 0.4]
ax.set_position(pos2)

plt.title('Coefficients of Linear Regression Model', y=1.17)
plt.suptitle('Adj. R-Squared of Model = 0.23 On Test Set', y=0.92)
plt.xlabel('Independant Variables')
plt.ylabel('Beta Coefficients')

plt.savefig('img/mvp_coefplot.png')

In [ ]:
# sns.PairGrid(X_train)

In [ ]:
xs = merged['release month']
ys = merged['decay rate']
plt.figure(figsize=(6,4))
plt.scatter(xs, ys)
plt.title('Decay Rates by Release Month')
plt.xlabel('Month')
plt.ylabel('Decay Rate')

ax = plt.gca()
pos1 = ax.get_position()
pos2 = [pos1.x0 + 0.02, pos1.y0 + 0.05,  pos1.width, pos1.height - 0.04]
ax.set_position(pos2)

plt.savefig('img/scatter_release_month.png')

In [ ]:
plt.figure(figsize=(4,4))
plt.scatter(y3_pred_test, results_3.resid)
plt.title('Fitted Y\'s vs. Residuals')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals of Model')

In [ ]:
plt.figure(figsize=(4,4))
resid3_density = gaussian_kde(results_3.resid)
xs = np.linspace(results_3.resid.min(),results_3.resid.max(),200)
ys = resid3_density(xs)
plt.plot(xs, ys)
plt.title('PDF of resids')
plt.ylabel('Probability')
plt.xlabel('Resids')

In [ ]:
from statsmodels.stats.diagnostic import het_white
het_white(results_3.resid, X3_test)

In [ ]:
# ittr3 = X3_train.iterrows()
# ypreds = []
# for row in ittr3:
#     y = results_3.predict(row)
#     ypreds.append(y)
# print y

## Scrape MetaCritic Data

In [2]:
def get_meta_rating_moviepg(soup):
    name_not_found = True
    for item in soup.find_all('span'): 
        if item.get('itemprop') == 'ratingValue':
            rating = item.text.strip()
        if item.get('itemprop') == 'name' and (name_not_found):
            title = item.text.strip()
            name_not_found = False
    return title, rating

In [5]:
# Tester for single movie page
meta_url = 'http://www.metacritic.com/movie/philadelphia'
soup = get_soup_meta(meta_url)
title, rating = get_meta_rating_moviepg(soup)
print title, rating

Philadelphia 66


/Users/MelanieAppleby/anaconda/envs/benson/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [4]:
def get_soup_url(url):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=hdr)
    page = response.text
    soup = BeautifulSoup(page, 'xml')
    return soup

def get_soup_meta(url):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = urllib2.Request(url,headers=hdr)
    page = urllib2.urlopen(req)
    soup = BeautifulSoup(page)
    return soup

# def get_soup_url(url):
#     pool = Pool(25)
#     pool.spawn(requests.get, url)
#     soup = BeautifulSoup(requests.get(url).text, 'html.parser')
#     return soup

def get_soups(url_list):
    pool = Pool(25)
    soup_list = []
    faulty_url_list = []
    for link in url_list:
        soup_list.append(pool.spawn(get_soup_url, link))
    pool.join()
    return soup_list

def get_meta_url_list():
    url_list = []
    meta_base = 'http://www.metacritic.com/browse/movies/title/dvd' 
    alpha = [''] + list(string.ascii_lowercase)
    for letter in alpha:
        url = meta_base + '/' + letter + '?view=condensed'
        url_list.append(url)
        pages = np.arange(1,20)
        for pg in pages: 
            url = meta_base + '/' + letter + '?view=condensed&page=' + str(pg)
            url_list.append(url)
    return url_list

In [7]:
# print len(url_list)
# half = len(url_list)/2
# print half

In [ ]:
# meta_ratings_dict = defaultdict(str)

# soup_list1 = get_soups(url_list[:half])
# print 'Done with first set'

In [ ]:
# soup_list2 = get_soups(url_list[half:])

In [ ]:
# print url_list[half:]

In [71]:
# print len(soup_list1)
# print len(soup_list2)

# empty_soups1 = [s for s in soup_list1 if s.value == None]
# print len(empty_soups1)
# empty_soups2 = [s for s in soup_list2 if s.value == None]
# print len(empty_soups2)

270
270
0
185


In [ ]:
# Rerun soup retrieval for second half, but in halves
# half2 = len(url_list[half:])/2
# print len(url_list[:half])
# print len(url_list[half:])
# print half2
# fake_url = 'http://www.metacritic.com/browse/movies/title/dvd/s?view=condensed&page=60'

# soup_list2_1 = get_soups(url_list[:half2])

In [ ]:
# print len(url_list)
# soup_test = get_soup_meta(url_list[100])
# print soup_test

In [8]:
url_list = get_meta_url_list()

soup_list = []
count = 1
for url in url_list:
    print 'Fetching data for soup: '+str(count)
    count += 1
    soup = get_soup_meta(url)
    soup_list.append(soup)

Fetching data for soup: 1
Fetching data for soup: 2
Fetching data for soup: 3
Fetching data for soup: 4
Fetching data for soup: 5
Fetching data for soup: 6
Fetching data for soup: 7
Fetching data for soup: 8
Fetching data for soup: 9
Fetching data for soup: 10
Fetching data for soup: 11
Fetching data for soup: 12
Fetching data for soup: 13
Fetching data for soup: 14
Fetching data for soup: 15
Fetching data for soup: 16
Fetching data for soup: 17
Fetching data for soup: 18
Fetching data for soup: 19
Fetching data for soup: 20
Fetching data for soup: 21
Fetching data for soup: 22
Fetching data for soup: 23
Fetching data for soup: 24
Fetching data for soup: 25
Fetching data for soup: 26
Fetching data for soup: 27
Fetching data for soup: 28
Fetching data for soup: 29
Fetching data for soup: 30
Fetching data for soup: 31
Fetching data for soup: 32
Fetching data for soup: 33
Fetching data for soup: 34
Fetching data for soup: 35
Fetching data for soup: 36
Fetching data for soup: 37
Fetching d

In [ ]:
# print len(soup_list)

In [11]:
import sys
sys.setrecursionlimit(3000)
with open('soup_list.pickle','wb') as handle:
    pickle.dump(soup_list, handle)

In [12]:
count = 1
meta_ratings_dict = {}

for soup in soup_list:
    print 'Getting data for soup ' + str(count)
    count += 1
    for movie in soup.find_all('div', class_='basic_stat product_score brief_metascore'):
        rating = movie.text.strip()
        parent = movie.find_parent('div')
        title = parent.find(class_='basic_stat product_title').text.strip()
        meta_ratings_dict[title] = rating

Getting data for soup 1
Getting data for soup 2
Getting data for soup 3
Getting data for soup 4
Getting data for soup 5
Getting data for soup 6
Getting data for soup 7
Getting data for soup 8
Getting data for soup 9
Getting data for soup 10
Getting data for soup 11
Getting data for soup 12
Getting data for soup 13
Getting data for soup 14
Getting data for soup 15
Getting data for soup 16
Getting data for soup 17
Getting data for soup 18
Getting data for soup 19
Getting data for soup 20
Getting data for soup 21
Getting data for soup 22
Getting data for soup 23
Getting data for soup 24
Getting data for soup 25
Getting data for soup 26
Getting data for soup 27
Getting data for soup 28
Getting data for soup 29
Getting data for soup 30
Getting data for soup 31
Getting data for soup 32
Getting data for soup 33
Getting data for soup 34
Getting data for soup 35
Getting data for soup 36
Getting data for soup 37
Getting data for soup 38
Getting data for soup 39
Getting data for soup 40
Getting d

In [15]:
with open('meta_ratings_dict.pickle','wb') as handle:
    pickle.dump(meta_ratings_dict, handle)

In [132]:
# print len(soup_list)
# print soup_list[0]

In [104]:
# print len(soup_list1)
# print len(soup_list2_1)
# print '~~~'
# empty_soups1 = [s for s in soup_list1 if s == None]
# print len(empty_soups1)
# empty_soups2 = [s for s in soup_list2_1 if s == None]
# print len(empty_soups2)

In [117]:
# print soup_list1[:10]
# print soup_list2_1[:10]
# soup_list_tot = soup_list1.extend(soup_list2_1)
# print soup_list_tot[:10]

In [13]:
# small = {k: meta_ratings_dict[k] for k in meta_ratings_dict.keys()[:10]}
# print small
print len(meta_ratings_dict)

7746


In [ ]:
def merge_dicts(*dict_args):
    '''
    Given any number of dicts, shallow copy and merge into a new dict,
    precedence goes to key value pairs in latter dicts.
    '''
    result = {}
    for dictionary in dict_args:
        result.update(dictionary)
    return result

In [16]:
ratings = pd.Series(meta_ratings_dict)
# for d in meta_ratings[:3]:
#     temp = pd.DataFrame(d.items(), columns=['movie title', 'meta_rating'])
#     ratings.append(temp)

In [22]:
ratings.head()

#Horror    42
$9.99      68
$pent      34
'71        83
'R Xmas    55
dtype: object

In [23]:
ratingsdf = pd.DataFrame(ratings, index=ratings.index, columns = ['meta_rating'])

In [26]:
ratingsdf.head()

,meta_rating
#Horror,42
$9.99,68
$pent,34
'71,83
'R Xmas,55


In [28]:
movie_data = pd.read_csv('csv/merged_table_4_20.csv')

In [30]:
movie_data.drop('Unnamed: 0', axis=1, inplace=True)

In [31]:
movie_data.head()

,distributor,domestic total gross,genre,movie title,movieid,number of theaters,opening weekend revenue,production budget,rating,release date,runtime (mins),time in theaters (days),release month,decay rate
0,Orion Pictures,90944400.0,Comedy,Dirty Rotten Scoundrels,1,1515.0,8129500.0,NaN,PG,1988-12-16,110.0,56.0,12.0,-0.171526
1,Fox,252358600.0,Action Thriller,Speed,2,2169.0,30088300.0,30000000.0,R,1994-06-10,116.0,70.0,6.0,-0.181872
2,Paramount,48884600.0,Comedy,The Out-of-Towners,3,2128.0,14084800.0,75000000.0,PG-13,1999-04-02,90.0,105.0,4.0,-0.169169
3,Universal,39263600.0,War Romance,Captain Corelli's Mandolin,4,1612.0,11081500.0,57000000.0,R,2001-08-17,129.0,63.0,8.0,-0.265453
4,Paramount,108640100.0,Comedy,"Planes, Trains and Automobiles",5,1684.0,15596500.0,NaN,R,1987-11-25,92.0,70.0,11.0,-0.189091


In [33]:
moviedf = pd.merge(movie_data, ratingsdf, left_on='movie title', right_index=True, how='left')

In [34]:
moviedf.head()

,distributor,domestic total gross,genre,movie title,movieid,number of theaters,opening weekend revenue,production budget,rating,release date,runtime (mins),time in theaters (days),release month,decay rate,meta_rating
0,Orion Pictures,90944400.0,Comedy,Dirty Rotten Scoundrels,1,1515.0,8129500.0,NaN,PG,1988-12-16,110.0,56.0,12.0,-0.171526,68
1,Fox,252358600.0,Action Thriller,Speed,2,2169.0,30088300.0,30000000.0,R,1994-06-10,116.0,70.0,6.0,-0.181872,78
2,Paramount,48884600.0,Comedy,The Out-of-Towners,3,2128.0,14084800.0,75000000.0,PG-13,1999-04-02,90.0,105.0,4.0,-0.169169,33
3,Universal,39263600.0,War Romance,Captain Corelli's Mandolin,4,1612.0,11081500.0,57000000.0,R,2001-08-17,129.0,63.0,8.0,-0.265453,36
4,Paramount,108640100.0,Comedy,"Planes, Trains and Automobiles",5,1684.0,15596500.0,NaN,R,1987-11-25,92.0,70.0,11.0,-0.189091,NaN


In [35]:
moviedf.count(axis=0)

distributor                3643
domestic total gross       3643
genre                      3643
movie title                3642
movieid                    3643
number of theaters         3616
opening weekend revenue    3242
production budget          1994
rating                     3643
release date               3643
runtime (mins)             3642
time in theaters (days)    3643
release month              3643
decay rate                 3643
meta_rating                2475
dtype: int64

In [36]:
moviedf_r = moviedf[pd.notnull(moviedf['meta_rating'])]
moviedf_r.head()

,distributor,domestic total gross,genre,movie title,movieid,number of theaters,opening weekend revenue,production budget,rating,release date,runtime (mins),time in theaters (days),release month,decay rate,meta_rating
0,Orion Pictures,90944400.0,Comedy,Dirty Rotten Scoundrels,1,1515.0,8129500.0,NaN,PG,1988-12-16,110.0,56.0,12.0,-0.171526,68
1,Fox,252358600.0,Action Thriller,Speed,2,2169.0,30088300.0,30000000.0,R,1994-06-10,116.0,70.0,6.0,-0.181872,78
2,Paramount,48884600.0,Comedy,The Out-of-Towners,3,2128.0,14084800.0,75000000.0,PG-13,1999-04-02,90.0,105.0,4.0,-0.169169,33
3,Universal,39263600.0,War Romance,Captain Corelli's Mandolin,4,1612.0,11081500.0,57000000.0,R,2001-08-17,129.0,63.0,8.0,-0.265453,36
6,DreamWorks,55006600.0,Comedy,Head of State,7,2256.0,19482600.0,NaN,PG-13,2003-03-28,95.0,112.0,3.0,-0.114673,44


In [38]:
moviedf_r.count(axis=0)

distributor                2475
domestic total gross       2475
genre                      2475
movie title                2475
movieid                    2475
number of theaters         2469
opening weekend revenue    2183
production budget          1703
rating                     2475
release date               2475
runtime (mins)             2474
time in theaters (days)    2475
release month              2475
decay rate                 2475
meta_rating                2475
dtype: int64

In [39]:
moviedf_r.to_csv('csv/moviedf_with_ratings.csv')

In [40]:
moviedf.to_csv('csv/moviedf_with_null_ratings.csv')